In [1]:
from robot_descriptions.a1_description import URDF_PATH
from darli.backend import PinocchioBackend, CasadiBackend, JointType
from darli.modeling import Robot, Functional
from darli.modeling.integrators import ForwardEuler, RK4, MidPoint
import numpy as np

In [2]:
dt = 1e-3
n_steps = 20
control_sampling = 5e-3
samples = int(dt * n_steps / control_sampling)

In [3]:
model = Functional(CasadiBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [4]:
np.random.seed(0)
tau = np.random.randn(model.nu)

tau

array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
       -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
        0.14404357,  1.45427351])

In [5]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

x0.shape, model.nq, model.nv

((37,), 19, 18)

## Functional way

In [6]:
euler_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

euler_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [7]:
euler_roll(x0, tau)

DM([6.86254e-05, -0.000313681, -0.00182676, -0.017466, -0.000772986, -9.55388e-05, 0.999847, 0.0469618, -0.0236709, 0.0713617, 0.0511848, 0.0695515, -0.148044, 0.04396, 0.0031811, -0.00541951, 0.0402101, -0.0454539, 0.123088, 0.00676575, -0.0263639, -0.197224, -3.62433, -0.152288, -0.0232693, 4.89491, -2.50181, 7.51287, 5.36967, 7.25888, -15.4562, 4.5822, 0.325292, -0.570832, 4.19973, -4.77632, 12.9125])

In [8]:
%timeit euler_roll()

302 µs ± 5.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
model.state_space.set_integrator(MidPoint)

midpoint_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

midpoint_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [10]:
midpoint_roll(x0, tau)

DM([7.17749e-05, -0.000326519, -0.00193021, -0.01834, -0.00080474, -0.000103313, 0.999831, 0.0493509, -0.0249361, 0.075124, 0.0538461, 0.0730974, -0.1556, 0.0461969, 0.00333232, -0.00570596, 0.04227, -0.0478277, 0.129481, 0.00670047, -0.0257048, -0.198048, -3.61433, -0.15033, -0.0238173, 4.8858, -2.50349, 7.5126, 5.36669, 7.24167, -15.4187, 4.57372, 0.323499, -0.570947, 4.19375, -4.77132, 12.8964])

In [11]:
%timeit midpoint_roll()

592 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
model.state_space.set_integrator(RK4)

rk4_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

rk4_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [13]:
%timeit rk4_roll()

1.19 ms ± 4.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Pinocchio way

In [14]:
model = Robot(PinocchioBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [15]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

np.random.seed(0)
tau = np.random.randn(model.nu, samples)

In [16]:
model.state_space.integrator

In [17]:
%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

187 µs ± 2.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
euler_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

euler_state, "quaternion norm", np.linalg.norm(euler_state[3:7])

(array([ 3.04277252e-04, -2.01330463e-04, -1.90248413e-03, -1.17040266e-02,
        -5.15787389e-04, -3.94341397e-04,  9.99931295e-01,  3.22966013e-02,
         1.50246560e-02, -1.98648484e-02,  2.99230958e-02,  5.13920733e-02,
        -1.17458362e-01,  2.93587807e-02,  5.98572029e-02, -1.30787434e-01,
         2.92957256e-02,  2.78096767e-04, -1.22598234e-02,  1.79929644e-02,
        -1.11581574e-02, -2.01115830e-01, -1.86671007e+00, -7.24765145e-02,
        -8.40182063e-02,  2.81446975e+00, -2.83148081e-01,  1.99502364e+00,
         2.41199628e+00,  2.77334598e+00, -6.48657322e+00,  2.17138080e+00,
         4.52850880e+00, -9.88360657e+00,  2.22627337e+00,  3.21107178e-01,
        -1.38804941e+00]),
 'quaternion norm',
 0.9999999999999999)

In [19]:
model.state_space.set_integrator(MidPoint)

%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

368 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
midpoint_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

midpoint_state, "quaternion norm", np.linalg.norm(midpoint_state[3:7])

(array([ 3.13056814e-04, -2.06007071e-04, -2.00579933e-03, -1.21762022e-02,
        -5.35876488e-04, -4.17895753e-04,  9.99925636e-01,  3.37227998e-02,
         1.48924239e-02, -1.88716407e-02,  3.11635841e-02,  5.27516271e-02,
        -1.20628686e-01,  3.04188482e-02,  6.21083248e-02, -1.35677435e-01,
         3.04083768e-02,  4.41604814e-04, -1.29512213e-02,  1.79570888e-02,
        -1.07038872e-02, -2.01332224e-01, -1.86844846e+00, -7.30573129e-02,
        -8.43885313e-02,  2.81685816e+00, -2.82157805e-01,  1.99486806e+00,
         2.41484678e+00,  2.77127195e+00, -6.48019797e+00,  2.17016381e+00,
         4.52686951e+00, -9.87708295e+00,  2.22696589e+00,  3.21654162e-01,
        -1.38766016e+00]),
 'quaternion norm',
 1.0)

In [21]:
model.state_space.set_integrator(RK4)

%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

768 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
model.state_space.model

In [23]:
rk4_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

rk4_state, "quaternion norm", np.linalg.norm(rk4_state[3:7])

(array([ 3.13037969e-04, -2.05839271e-04, -2.00601165e-03, -1.21761337e-02,
        -5.35736848e-04, -4.18077826e-04,  9.99925637e-01,  3.37230957e-02,
         1.48924051e-02, -1.88717933e-02,  3.11656303e-02,  5.27487210e-02,
        -1.20622606e-01,  3.04167959e-02,  6.21072024e-02, -1.35674861e-01,
         3.04075968e-02,  4.41164010e-04, -1.29509577e-02,  1.79563270e-02,
        -1.06986147e-02, -2.01343010e-01, -1.86843671e+00, -7.30281345e-02,
        -8.44036179e-02,  2.81684357e+00, -2.82184416e-01,  1.99487844e+00,
         2.41494885e+00,  2.77110956e+00, -6.47989044e+00,  2.17006990e+00,
         4.52679842e+00, -9.87696459e+00,  2.22693353e+00,  3.21613763e-01,
        -1.38764357e+00]),
 'quaternion norm',
 1.0)